In [1]:
import os

In [2]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one\\Experimentation'

WE NEED TO GO ONE DIRECTORY BACK

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one'

In [5]:
#  ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)      # you don't have to give self in a dataclass CLASS
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from portfolio_project_one.constants import *
from portfolio_project_one.utils.common import read_yaml, create_directories

In [7]:
#  Updating Configuratiom Manager inside src/ config

class ConfigurationManager:                  
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,                     # These were all defined in constants
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifact_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:   # return type is the entity we created

        config = self.config.data_ingestion

        create_directories([config.root_dir])     # we did define the root directory but i never had such a directory,  this is solved here

        data_ingestion_configuration = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_configuration

In [15]:
#  Updating Components

import os
import urllib.request as request    # to download the file from the url
import zipfile
from portfolio_project_one import logger
from portfolio_project_one.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            print(self.config.source_URL)
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,                               # Thw link gave me an error because my repo was private
                filename = self.config.local_data_file
            )

            logger.info(f'{filename} downloaded successfully!, with following info: \n{headers}')
        else:
            logger.info(f'File already exists pf size: {get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):
        '''
        This will unzip the downloaded zip file
        '''
        zip_extract_to_path = self.config.unzip_dir
        os.makedirs(zip_extract_to_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(zip_extract_to_path)
            
     

In [17]:
#  Update Pipeline
try: 
    config = ConfigurationManager()
    data_ingeston_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingeston_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-10-06 19:28:05,621: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-06 19:28:05,623: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-06 19:28:05,626: INFO: common: yaml file: schema.yaml loaded succesfully]
[2023-10-06 19:28:05,628: INFO: common: Created directory at : artifacts]
[2023-10-06 19:28:05,630: INFO: common: Created directory at : artifacts/data_ingestion]
https://github.com/swarajbari18/DATA/raw/main/winequality-data.zip
[2023-10-06 19:28:06,919: INFO: 3093719100: artifacts/data_ingestion/data.zip downloaded successfully!, with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub